In [1]:
import ipywidgets as widgets
from ipywidgets import Button, Layout
from IPython.display import display
import matplotlib.pyplot as plt
import webbrowser
from ciboulette.base import ciboulette
from ciboulette.utils import exposure
from ciboulette.indiclient.camera import QHY5MCam

In [104]:
camvisu = ciboulette.Ciboulette()
camvisu.observer_name = 'CAMVISU'
camvisu.qhy5M
exp = exposure.Exposure()
exp.exp_time = 15
Latency = 2.4
Gain = 35
camvisu.exposure = exp

In [8]:
while True:
    try:
        qhy5M = QHY5MCam('192.168.1.30',7628)
        error = ''
        break
    except:
        error = 'Noconnect'
        break

In [61]:
output = widgets.Output(layout={'width': '100%'})
output

Output(layout=Layout(width='100%'))

In [63]:
outputfits = widgets.Output(layout={'width': '100%'})
outputfits

Output(layout=Layout(width='100%'))

In [64]:
def on_go_clicked(b):
    with outputfits:
        outputfits.clear_output()
        exp.now2label
        qhy5M.gain = Gain
        camvisu.exposure = exp
        hdul = qhy5M.expose(camvisu.exposure[0],latency=Latency)
        plt.figure(figsize=(15,15))
        plt.gca().axis('off')
        plt.imshow(hdul[0].data, origin='lower', cmap='gray',vmin = 0, vmax = 100)
        plt.show()
        hdul.close()
        # take the fits

In [102]:
def on_start_clicked(b):
    with outputfits:
        while textnumber.value > 0 :
            exp.now2label
            qhy5M.gain = Gain
            camvisu.exposure = exp
            hdul = qhy5M.expose(camvisu.exposure[0],latency=Latency)
            outputfits.clear_output()
            plt.figure(figsize=(15,15))
            plt.gca().axis('off')
            plt.imshow(hdul[0].data, origin='lower', cmap='gray',vmin = 0, vmax = 100)
            plt.show()
            hdr = camvisu.extendedhdr(hdul[0].header)
            hdul[0].header = hdr
            filename = camvisu.observer_name + str(camvisu.exposure[1]) + '.fits'
            hdul.writeto('dataset/'+filename)
            hdul.close()
            textnumber.value -= 1
            # save the fits

In [105]:
if error != 'Noconnect' :
    buttongo = widgets.Button(description="Go Visu",layout=Layout(width='100%'))
    buttonstart = widgets.Button(description="Start",layout=Layout(width='50%'))
    textnumber = widgets.IntText(value=60,description='Nunber:',disabled=False,layout=Layout(width='50%'))
    buttongo.on_click(on_go_clicked)
    buttonstart.on_click(on_start_clicked)
    
    with output:
        output.clear_output()
        display(widgets.HBox([buttongo], width='100%'), widgets.HBox([textnumber, buttonstart]), width='100%')
else :
     with output:
        output.clear_output()
        display(print('Noconnect'))